In [1]:
import numpy as np 
import pandas as pd 
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from scipy import sparse
import random
import lightfm 
from lightfm import LightFM, cross_validation
from lightfm.evaluation import precision_at_k, recall_at_k, auc_score, reciprocal_rank
from sklearn.metrics.pairwise import cosine_similarity
# inspired by https://www.kaggle.com/code/pegahpooya/spotify-playlists-recommender-system/notebook

#p = 1.0 
df = pd.read_csv('purchase_history_anonymized.csv', sep=',', error_bad_lines=False, warn_bad_lines=False) #, skiprows=lambda i: i>0 and random.random() > p)
df = df.groupby('product_L2').filter(lambda x : len(x)>=30)
df = df[df.groupby('customer').product_L2.transform('nunique') >= 10] # 10 interactions with unique products
customer_counts = df.groupby('customer')['product_L2'].count()
customers_with_20_or_more_products = customer_counts[customer_counts >= 20].index # 20 interactions in total
df = df[df['customer'].isin(customers_with_20_or_more_products)]

customer_id = 'cust-00008'
customer_interactions = df[df['customer'] == customer_id]
last_5_interactions = customer_interactions.drop_duplicates(subset='product_L2', keep='last').tail(5)
product_ids = last_5_interactions['product_L2'].tolist()
df_siemens = df[~((df['customer'] == customer_id) & (df['product_L2'].isin(product_ids)))]

size = lambda x: len(x)
df_freq = df_siemens.groupby(['customer', 'product_L2']).agg('size').reset_index().rename(columns={0:'freq'})[['customer', 'product_L2', 'freq']].sort_values(['freq'], ascending=False)

df_productL2 = pd.DataFrame(df_freq["product_L2"].unique())
df_productL2 = df_productL2.reset_index()
df_productL2 = df_productL2.rename(columns={'index':'product_L2_id', 0:'product_L2'})

df_freq  = pd.merge(df_freq , df_productL2, how='inner', on='product_L2')

unique_customers = df['customer'].nunique()
number_of_rows = df.shape[0]
print(f"unique customers: {unique_customers}")
print(f"Number of rows in the DataFrame: {number_of_rows}")

unique customers: 4577
Number of rows in the DataFrame: 1967994


In [2]:
def create_interaction_matrix(df,user_col, item_col, rating_col, norm= False, threshold = None):
    interactions = df.groupby([user_col, item_col])[rating_col] \
            .sum().unstack().reset_index(). \
            fillna(0).set_index(user_col)
    if norm:
        interactions = interactions.applymap(lambda x: 1 if x > threshold else 0)
    return interactions

def create_user_dict(interactions):
    customer = list(interactions.index)
    user_dict = {}
    counter = 0 
    for i in customer:
        user_dict[i] = counter
        counter += 1
    return user_dict

def create_item_dict(df,id_col,name_col):
    item_dict ={}
    for i in range(df.shape[0]):
        item_dict[(df.loc[i,id_col])] = df.loc[i,name_col]
    return item_dict


def accuracy_at_k(model, interactions, k):

    interactions_dense = interactions.toarray()
    predictions = model.predict_rank(interactions)
    predictions_dense = predictions.toarray()
    relevant = interactions_dense > 0
    selected = predictions_dense < k
    accuracy_per_user = np.mean(np.logical_or(np.logical_not(relevant), selected), axis=1)

    return np.mean(accuracy_per_user)


def runMF(interactions, n_components=700, loss='warp-kos', k=15, epoch=30,n_jobs = 4, max_sampled= 3): #n_components can be change for better performance   
    model = LightFM(no_components= n_components, loss=loss,k=k, max_sampled=max_sampled)
    model.fit(x,epochs=epoch,num_threads = n_jobs)
    return model

def sample_recommendation_user(model, interactions, customer, user_dict, item_dict,threshold = 0,nrec_items = 10, show = True):
    n_users, n_items = interactions.shape
    user_x = user_dict[customer]
    scores = pd.Series(model.predict(user_x,np.arange(n_items)))
    scores.index = interactions.columns
    scores = list(pd.Series(scores.sort_values(ascending=False).index))
    
    known_items = list(pd.Series(interactions.loc[customer,:] \
                                 [interactions.loc[customer,:] > threshold].index) \
								 .sort_values(ascending=False))
    
    scores = [x for x in scores if x not in known_items]
    return_score_list = scores[0:nrec_items]
    known_items = list(pd.Series(known_items).apply(lambda x: item_dict[x]))
    scores = list(pd.Series(return_score_list).apply(lambda x: item_dict[x]))
    if show == True:
        print("Known Likes:")
        counter = 1
        for i in known_items:
            print(str(counter) + '- ' + i)
            counter+=1

        print("\n Recommended Items:")
        counter = 1
        for i in scores:
            print(str(counter) + '- ' + i)
            counter+=1
    return return_score_list

In [3]:
interactions = create_interaction_matrix(df = df_freq, user_col = "customer", item_col = 'product_L2_id', rating_col = 'freq', norm= False, threshold = None)
interactions.head()
user_dict = create_user_dict(interactions=interactions)
product_L2s_dict = create_item_dict(df = df_productL2, id_col = 'product_L2_id', name_col = 'product_L2')

x = sparse.csr_matrix(interactions.values)
train, test = lightfm.cross_validation.random_train_test_split(x, test_percentage=0.3, random_state=None)
model = runMF(interactions = train, n_components = 700, loss = 'warp-kos', k = 15, epoch = 30, n_jobs = 4, max_sampled=3)
train_auc = auc_score(model, train, num_threads=4).mean()
test_auc = auc_score(model, test, train_interactions=train, num_threads=4).mean()
train_precision = precision_at_k(model, train, k=10).mean()
test_precision_at_10 = precision_at_k(model, test, k=10, train_interactions=train).mean()
test_precision_at_5 = precision_at_k(model, test, k=5, train_interactions=train).mean()
test_recall = recall_at_k(model, test, k=10, train_interactions=train).mean()
test_recall_at_20 = recall_at_k(model, test, k=20, train_interactions=train).mean()
test_recall_at_40= recall_at_k(model, test, k=40, train_interactions=train).mean()
test_reciprocal = reciprocal_rank(model, test, train_interactions=train, num_threads=4).mean()

In [4]:
print('Test AUC: %.7f' % (test_auc))
print('Test Reciprocal: %.3f' % (test_reciprocal))
print('Test Precision at 10 %.3f' % (test_precision_at_10))
print('Test Precision at 5 %.3f' % (test_precision_at_5))
print('Test Recall at 10 %.3f' % (test_recall))
print('Test Recall at 20 %.3f' % (test_recall_at_20))

Test AUC: 0.9999406
Test Reciprocal: 0.996
Test Precision at 10 0.804
Test Precision at 5 0.938
Test Recall at 10 0.752
Test Recall at 20 0.910


## Metric: Precision at k
##### k = 10 
Out of k recommended items, the user interacted with x items – we will treat this as a binary signal of relevance. In this case, the Avg Precision at 10 is 80,4%. Out of 10 recommendations, more than 8 were good ones. The system is doing a decent job. 
##### k = 5
In this case, the Avg Precision at 5 is 93,8%. Out of 5 recommendations, the avg is almost 5. We can  definitely say that the first 5 recommendations look very promising. 

## Metric: Recall at k
Imagine you have a list of top k recommendations, and there are a total of 30 items in the dataset that are actually relevant (only an assumption). If the system includes 9 relevant items in the top 10, the Recall at 10 is 30% (9 out of 30).

## Metric: Reciprocal ranking
Mean Reciprocal Rank (MRR) is a ranking quality metric. It considers the position of the first relevant item in the ranked list. A Reciprocal Rank is the inverse of the position of the first relevant item. If the first relevant item is in position 2, the reciprocal rank is 1/2. If we think of recommendations for 6 users with the following example values: 
- For user 1, the first relevant item is in position 1, so the RR is 1. 
- For user 2,4,6, the first relevant item is in position 3, so the RR is 1/3 ≈ 0.33. 
- For user 3, the first relevant item is in position 2, so the RR is 1/2 = 0,5. 
- For user 5, the first relevant item is in position 4, so the RR is 1/4 = 0.25. 
#### Overall: MRR = (1 + 0,33 + 0,5 + 0,33 + 0,25 + 0,33) = 2,74/6 = 0,456

####  The system gets a Reciprocal Ranking of 0.996 
This indicates that the first relevant item is almost everytime the first recommended products overall, which is very pleasant result.

- AUC             => 0.999
- Precision at 10 => 80,4% of the top 10 are relevant items
- Precision at 5  => 93,8% of the top 5 are relevant items
- Recall at 10    => 75,2% of all relevant items are in the top 10 
- Recall at 20    => 91,0% of all relevant items are in the top 20 

Those are decent numbers for arecommender system, which shows a overall good performance.


### Another evaluation technique - Hitrate for a single user

In [5]:
print(last_5_interactions)

         date  country    customer industry product_L1  product_L2  \
2023995  1490  land-12  cust-00008   ind-42  prdL1-063  prdL2-1339   
2024153  1490  land-12  cust-00008   ind-42  prdL1-068  prdL2-1543   
2031265  1497  land-12  cust-00008   ind-42  prdL1-060  prdL2-1131   
2032669  1497  land-12  cust-00008   ind-42  prdL1-063  prdL2-1344   
2033793  1498  land-12  cust-00008   ind-42  prdL1-061  prdL2-1158   

          product_L3  price_rescaled  
2023995  prdL3-25218             302  
2024153  prdL3-30015            2552  
2031265  prdL3-23728             145  
2032669  prdL3-25245            1165  
2033793  prdL3-23814              40  


In [6]:
# generating recommendations
rec_list = sample_recommendation_user(model = model, 
                                      interactions = interactions, 
                                      customer = 'cust-00008', 
                                      user_dict = user_dict,
                                      item_dict = product_L2s_dict, 
                                      threshold = 0,
                                      nrec_items = 5,
                                      show = True)

Known Likes:
1- prdL2-1184
2- prdL2-1182
3- prdL2-1155
4- prdL2-2108
5- prdL2-1598
6- prdL2-1822
7- prdL2-1620
8- prdL2-1014
9- prdL2-0916
10- prdL2-1427
11- prdL2-0960
12- prdL2-1263
13- prdL2-1700
14- prdL2-1599
15- prdL2-1837
16- prdL2-1245
17- prdL2-1180
18- prdL2-1336
19- prdL2-1160
20- prdL2-1574
21- prdL2-1159
22- prdL2-1329
23- prdL2-1830
24- prdL2-1178
25- prdL2-1334
26- prdL2-1823
27- prdL2-1134
28- prdL2-1276
29- prdL2-1836
30- prdL2-1170
31- prdL2-1592
32- prdL2-1345
33- prdL2-1223
34- prdL2-1235
35- prdL2-0964
36- prdL2-1358
37- prdL2-1332
38- prdL2-1835
39- prdL2-1219
40- prdL2-1762
41- prdL2-1839
42- prdL2-1794
43- prdL2-1244
44- prdL2-1238
45- prdL2-1239
46- prdL2-1340
47- prdL2-1356
48- prdL2-1396
49- prdL2-1240
50- prdL2-1208
51- prdL2-0911
52- prdL2-1234
53- prdL2-1423
54- prdL2-1221
55- prdL2-1243
56- prdL2-1587
57- prdL2-1428
58- prdL2-1201
59- prdL2-0194
60- prdL2-1547
61- prdL2-0912
62- prdL2-1202
63- prdL2-1204
64- prdL2-1132
65- prdL2-0195
66- prdL2-1825
67- pr

### 2/5 are in the Top 5: L2-1339 (Top 1), L2-1344 (Top 5)

### the deleted interactions, claims 2 top 5 spots
The interactions were deleted before training the model, so that the model treats these products as if they had never been purchased before. This looks pretty promising, especially because 2 of the 5 deleted items, claims a top 5 spot 

### Item-Item Recommendations
LightFM is primarily designed for user-item interactions, i.e. it focuses on predicting interactions between users and items based on previous interactions. To obtain item-item collaboration recommendations, we need to generate recommendations that match a specific item. We can analyse the relationships between items based on user interactions. This can be achieved by looking at which items are often purchased together or consumed by the same users. This approach is often referred to as "co-operative filtering".

In [7]:
def create_item_cooccurrence_matrix(interactions):
    
    cooccurrence_matrix = interactions.T.dot(interactions).toarray()
    np.fill_diagonal(cooccurrence_matrix, 0)  # 0 to avoid self-interaction
    return cooccurrence_matrix

cooccurrence_matrix = create_item_cooccurrence_matrix(x)

def get_item_recommendations(item_id, cooccurrence_matrix, item_dict, top_k=10):
    
    item_cooccurrences = cooccurrence_matrix[:, item_id] # extracting the co-interactions for the item
    top_items = np.argsort(-item_cooccurrences)[:top_k] # find the top items with the most co-interactions
    
    recommended_items = [item_dict[item] for item in top_items] # translating the item-iDs 
    return recommended_items

recommended_items = get_item_recommendations(2, cooccurrence_matrix, product_L2s_dict, top_k=10)

print("Recommended items for this purchase:", recommended_items)

Recommended items for this purchase: ['prdL2-1228', 'prdL2-1204', 'prdL2-1202', 'prdL2-1201', 'prdL2-1132', 'prdL2-1423', 'prdL2-0912', 'prdL2-1587', 'prdL2-1131', 'prdL2-1206']


This is a basic approach, but it does the job. However, methods such as ARM and ALS are likely to perform better.